In [2]:
import openai
import pinecone


from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_pinecone import PineconeVectorStore
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain

from pinecone import Pinecone, ServerlessSpec

# Setting Up 

In [4]:
#loading environment variables 
load_dotenv("api.env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_API_ENV = os.getenv('PINECONE_API_ENV')

In [5]:
#Set model 
model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo", temperature=0)

# Loading and Splitting Data

In [6]:
#Load master_pcos txt file 
loader = TextLoader("Guidelines (txt)/master_pcos.txt")
data = loader.load()

#Split document into chunks, put into documents variable
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
master_documents = text_splitter.split_documents(data)

In [7]:
len(master_documents)

219

In [8]:
#Embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=OPENAI_API_KEY)

# Vector Store

In [14]:
index_name = "basic"

#Commented out so that master_docs isn't loaded into vector store multiple times
pinecone = PineconeVectorStore.from_documents(master_documents, embeddings, index_name=index_name)

# RetrievalQA

**No memory of previous conversation**

types of chains: https://ogre51.medium.com/types-of-chains-in-langchain-823c8878c2e9

**"stuff" chain type used** -> to save cost, otherwise would need to use a lot of tokens -> especially problematic once we start scaling! 

In [15]:
#Define retriever and QA chain 
retriever = pinecone.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=model, 
    chain_type="stuff", 
    retriever=retriever, 
    return_source_documents=False
)

In [16]:
#Defining function for user to ask LLM question 
def ask_question():
    # Prompt the user to input a question
    user_question = input("Ask a PCOS-related question: ")
    result = qa({"query": user_question})
    print("Answer:", result['result'])

In [17]:
ask_question()

Ask a PCOS-related question: At what age range can androgen levels in adolescents be expected to reach adult ranges for PCOS assessment?


/Users/Misa/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Answer: Androgen levels in adolescents can be expected to reach adult ranges typically between 12-15 years of age.


# Conversational Chain

**QA Chain + Conversation Memory**

In [18]:
qa_memory = ConversationalRetrievalChain.from_llm(
    llm=model, 
    chain_type="stuff", 
    retriever=retriever, 
    return_source_documents=False
)

In [19]:
def interactive_qa_session():
    print("Welcome to the PCOS Information Assistant. Type 'exit' to quit.")
    chat_history = []  # Initialize chat history

    while True:
        query = input("Ask a question about PCOS: ")
        if query.lower() == 'exit':
            print("Exiting the PCOS Information Assistant. Thank you!")
            break

        # Pass the question and the chat history to the ConversationalRetrievalChain
        result = qa_memory({"question": query, "chat_history": chat_history})
        print("Answer:", result['answer'])
        # Update chat history with the current query and answer
        chat_history.append((query, result['answer']))

In [20]:
interactive_qa_session()

Welcome to the PCOS Information Assistant. Type 'exit' to quit.
Ask a question about PCOS: Discuss the role of cardiovascular disease risk assessment in the management of PCOS.
Answer: Cardiovascular disease risk assessment plays a crucial role in the management of Polycystic Ovary Syndrome (PCOS). Women with PCOS are considered to be at an increased risk of cardiovascular disease and potentially cardiovascular mortality. Therefore, it is recommended that all women with PCOS should be assessed for cardiovascular disease risk factors. This includes having a lipid profile (cholesterol, LDL, HDL, and triglyceride levels) at the time of diagnosis, with subsequent measurements based on the presence of hyperlipidemia and additional risk factors.

Regular monitoring of blood pressure is also essential for women with PCOS, especially when planning pregnancy or seeking fertility treatment, due to the high risk of hypertensive disorders in pregnancy. Additionally, healthcare professionals should

Ask a question about PCOS: What are the best practices for family and partner involvement in the psychological care of PCOS patients?
Answer: The best practices for family and partner involvement in the psychological care of PCOS patients include:

1. Providing social and family support: Family and peer support can reduce stress, increase confidence, and provide a sense of security for PCOS patients.

2. Understanding cultural variations: Recognizing cultural differences and providing culturally appropriate resources and education can enhance the support provided by family and partners.

3. Encouraging shared decision making: Involving family members and partners in shared decision making can help in managing the psychological aspects of PCOS effectively.

4. Prioritizing patient agency: Supporting patient agency or the ability to take independent actions to manage their health and care is crucial in psychological care.

5. Increasing societal awareness and education: Public health act

Ask a question about PCOS: Are there specific micronutrients or supplements that should be considered in the diet of women with PCOS?
Answer: I don't have specific information on micronutrients or supplements for women with PCOS in the provided context. It is recommended to consult with a healthcare professional or a registered dietitian who is knowledgeable about PCOS to get personalized advice on specific nutrients or supplements that may be beneficial for managing PCOS.
Ask a question about PCOS: What are the emerging trends in nutrigenomics that could impact dietary recommendations for PCOS?
Answer: I don't have information on emerging trends in nutrigenomics that could impact dietary recommendations for PCOS based on the provided context.
Ask a question about PCOS: What are the first-line pharmacological treatments for managing menstrual irregularity in PCOS?
Answer: The first-line pharmacological treatments for managing menstrual irregularity in PCOS include the combined oral con

Ask a question about PCOS: Are there any advancements in nano-medicine that could improve drug delivery and effectiveness in infertility treatments for PCOS?
Answer: I don't have information on specific advancements in nano-medicine related to drug delivery for infertility treatments in PCOS. It's a rapidly evolving field, so it would be best to consult recent research articles or experts in the field for the most up-to-date information on this topic.
Ask a question about PCOS: What role do stem cells play in potential future treatments for infertility in PCOS patients?
Answer: I don't have information on the role of stem cells in potential future treatments for infertility in PCOS patients based on the provided context.
Ask a question about PCOS: What specific genetic polymorphisms in PCOS patients are linked to infertility, and how can targeted therapies be developed based on these findings?
Answer: I don't have information on specific genetic polymorphisms in PCOS patients that are 

Ask a question about PCOS: exit
Exiting the PCOS Information Assistant. Thank you!
